In [1]:
import pandas as pd
import numpy as np
import json
import re 
import sys
import itertools

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras import utils

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, accuracy_score

import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows", None)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [26]:
spotify_df = pd.read_csv('/content/drive/My Drive/projects/spotifyDataset.csv', nrows=5000)

In [27]:
spotify_df.isna().sum()

Unnamed: 0          0
track_id            0
artists             0
album_name          0
track_name          0
popularity          0
duration_ms         0
explicit            0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
time_signature      0
track_genre         0
dtype: int64

In [28]:
spotify_df.head()

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


In [30]:
mood_prep = spotify_df[['duration_ms', 'danceability', 'acousticness', 'energy', 'instrumentalness',
       'liveness', 'valence', 'loudness', 'speechiness', 'tempo']]

In [31]:
col_features = mood_prep.columns[:]
col_features

Index(['duration_ms', 'danceability', 'acousticness', 'energy',
       'instrumentalness', 'liveness', 'valence', 'loudness', 'speechiness',
       'tempo'],
      dtype='object')

In [32]:
mood_trans = MinMaxScaler().fit_transform(mood_prep[col_features])
mood_trans_np = np.array(mood_prep[col_features])

In [34]:
mood_trans_np[1011]

array([ 1.8000e+05,  7.0800e-01,  5.0700e-02,  7.8200e-01,  0.0000e+00,
        2.5100e-01,  7.1700e-01, -6.4190e+00,  2.8400e-01,  9.4038e+01])

In [35]:
df = pd.read_csv('/content/drive/My Drive/projects/data_moods.csv')

In [36]:
cl_features = df.columns[6:-3]
X= MinMaxScaler().fit_transform(df[cl_features])
X2 = np.array(df[cl_features])
Y = df['mood']

In [37]:
encoder = LabelEncoder()
encoder.fit(Y)
encoded_y = encoder.transform(Y)


dummy_y = utils.to_categorical(encoded_y)

X_train,X_test,Y_train,Y_test = train_test_split(X,encoded_y,test_size=0.2,random_state=15)

target = pd.DataFrame({'mood':df['mood'].tolist(),'encode':encoded_y}).drop_duplicates().sort_values(['encode'],ascending=True)
target

,mood,encode
5,Calm,0
4,Energetic,1
0,Happy,2
1,Sad,3


In [38]:
def base_model():
    model = Sequential()
    model.add(Dense(8,input_dim=10,activation='relu'))
    model.add(Dense(4,activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam',
                 metrics=['accuracy'])
    return model

In [39]:
#Configure the model
estimator = KerasClassifier(build_fn=base_model,epochs=300,batch_size=200,verbose=0)

In [40]:
#Evaluate the model using KFold cross validation
kfold = KFold(n_splits=10,shuffle=True)
results = cross_val_score(estimator,X,encoded_y,cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100,results.std()*100))

Baseline: 78.27% (3.27%)


In [20]:
estimator.fit(X_train,Y_train)
y_preds = estimator.predict(X_test)

5/5 [==============================] - 0s 3ms/step


In [41]:
y_preds

array([1, 0, 0, 1, 3, 2, 0, 2, 2, 1, 1, 2, 1, 0, 0, 3, 3, 1, 3, 2, 0, 0,
       3, 0, 1, 3, 0, 3, 3, 2, 1, 1, 0, 1, 3, 0, 2, 1, 0, 3, 1, 0, 0, 3,
       3, 3, 0, 2, 1, 0, 2, 1, 3, 0, 1, 3, 1, 1, 1, 1, 1, 0, 3, 0, 3, 1,
       2, 2, 0, 3, 1, 3, 3, 0, 0, 0, 3, 3, 1, 0, 3, 2, 1, 1, 0, 1, 3, 3,
       0, 1, 1, 0, 1, 2, 3, 1, 1, 0, 3, 1, 1, 0, 0, 3, 3, 2, 2, 0, 1, 2,
       0, 3, 0, 0, 2, 1, 2, 3, 3, 0, 3, 0, 2, 0, 1, 0, 0, 0, 0, 1, 3, 3,
       1, 1, 1, 3, 3, 2])

In [42]:
#Join the model and the scaler in a Pipeline
pip = Pipeline([('minmaxscaler',MinMaxScaler()),('keras',KerasClassifier(build_fn=base_model,epochs=300, batch_size=200,verbose=0))])
#Fit the Pipeline
pip.fit(X2,encoded_y)

Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('keras',
                 <keras.wrappers.scikit_learn.KerasClassifier object at 0x7f34172ac220>)])

In [43]:
def predict_mood(preds):

    preds_features = np.array(preds[:]).reshape(-1,1).T

    #Predict the features of the song
    results = pip.predict(preds_features)

    mood = np.array(target['mood'][target['encode']==int(results)])

    return str(mood[0])
    #print(f"{name_song} by {artist} is a {mood[0].upper()} song")
    

In [44]:
res = []

for i in range(len(mood_trans_np)):
  res.append(predict_mood(mood_trans_np[i]))

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 64ms/step


In [45]:
spotify_df.shape

(5000, 21)

In [46]:
print(len(res))

5000


In [47]:
spotify_df['Mood'] = np.resize(res,len(spotify_df))

In [54]:
spotify_df.to_csv('MusicMood.csv')

In [49]:
res.count("Sad")

1667

In [50]:
res.count("Happy")

1614

In [51]:
res.count("Energetic")

1024

In [52]:
res.count("Calm")

695

In [55]:
from google.colab import files
files.download('MusicMood.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>